In [1]:
!pip install transformers==4.28.0
!pip install datasets
!pip install seqeval

In [2]:
import os
import itertools
import pandas as pd
import numpy as np
from datasets import Dataset
from datasets import load_metric
from transformers import AutoTokenizer
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
from transformers import DataCollatorForTokenClassification
import torch

In [3]:
label_set = set()
tokens = []
pos_tags = []
token = []
pos_tag = []
with open('data/pkfn_clean_accumulated.tsv') as file:
  for line in file:
    line = line.strip()    
    if len(line) == 0:
      tokens.append(token)
      pos_tags.append(pos_tag)
      token = []
      pos_tag = []
    elif line.startswith("#"):
      continue
    else:
      words = line.split("\t")
      label_set.add(words[-1])
      token.append(words[1])
      pos_tag.append(words[-1])

label_list = list(label_set)

label_encoding_dict = {}
for i in range(len(label_list)):
  label_encoding_dict[label_list[i]] = i

train_df = pd.DataFrame({'tokens':tokens, 'pos_tags': pos_tags})

In [4]:
train_df.iloc[0]["tokens"]

['극지방,',
 '고산지대',
 '등의',
 '빙하와',
 '빙산이',
 '유례없이',
 '빠른',
 '속도로',
 '녹고',
 '있어',
 '지구상에',
 '큰',
 '환경',
 '및',
 '기후',
 '변화를',
 '초래할',
 '것이라고',
 '환경감시기구인',
 '월드워치',
 '인스티튜트가',
 '6',
 '일',
 '경고했다.']

In [5]:
train_df.iloc[0]["pos_tags"]

['B-Cause',
 'I-Cause',
 'I-Cause',
 'I-Cause',
 'I-Cause',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-Place',
 'B-Effect',
 'I-Effect',
 'I-Effect',
 'I-Effect',
 'I-Effect',
 'V',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [6]:
len(train_df["tokens"])

1766

In [7]:
len(train_df["pos_tags"])

1766

In [8]:
tokens = []
pos_tags = []
token = []
pos_tag = []
with open('data/pkfn_clean_accumulated.tsv') as file:
  for line in file:
    line = line.strip()    
    if len(line) == 0:
      tokens.append(token)
      pos_tags.append(pos_tag)
      token = []
      pos_tag = []
    elif line.startswith("#"):
      continue
    else:
      words = line.split("\t")
      label_set.add(words[-1])
      token.append(words[1])
      pos_tag.append(words[-1])

dev_df = pd.DataFrame({'tokens':tokens, 'pos_tags': pos_tags})

train_dataset = Dataset.from_pandas(train_df)
dev_dataset = Dataset.from_pandas(dev_df)

In [9]:
def tokenize_and_align_labels(examples):
    label_all_tokens = True
    tokenized_inputs = tokenizer(list(examples["tokens"]), truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"{task}_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif label[word_idx] == '0':
                label_ids.append(0)
            elif word_idx != previous_word_idx:
                label_ids.append(label_encoding_dict[label[word_idx]])
            else:
                label_ids.append(label_encoding_dict[label[word_idx]] if label_all_tokens else -100)
            previous_word_idx = word_idx
        labels.append(label_ids)
        
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
task = "pos" 
batch_size = 16

In [12]:
# ### DIDN'T work
# # # !pip install datsets transformers[sentencepiece]
# # # !pip install sentencepiece
# # from transformers import T5Tokenizer, T5ForConditionalGeneration
# # tokenizer = T5Tokenizer.from_pretrained(model_checkpoint)

# from transformers import AutoModel, AutoTokenizer
model_checkpoint = "snunlp/KR-BERT-char16424"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [14]:
train_dataset.features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'pos_tags': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None)}

In [15]:
# from transformers import AutoTokenizer, AutoModelForCausalLM 

# model_checkpoint = "kakaobrain/kogpt"
# tokenizer = AutoTokenizer.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   bos_token='[BOS]', eos_token='[EOS]', unk_token='[UNK]', pad_token='[PAD]', mask_token='[MASK]')


In [15]:
train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
dev_tokenized_datasets = dev_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/1766 [00:00<?, ? examples/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Map:   0%|          | 0/1766 [00:00<?, ? examples/s]

In [20]:
train_tokenized_datasets

Dataset({
    features: ['tokens', 'pos_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
    num_rows: 1766
})

In [17]:
!pip install accelerate #   low_cpu_mem_usage=True 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.1/219.1 kB 14.7 MB/s eta 0:00:00


In [18]:
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

# model = AutoModelForCausalLM.from_pretrained(
#   'kakaobrain/kogpt', revision='KoGPT6B-ryan1.5b-float16',  # or float32 version: revision=KoGPT6B-ryan1.5b
#   pad_token_id=tokenizer.eos_token_id,
#   num_labels=len(label_list),
#   torch_dtype='auto',
#   low_cpu_mem_usage=True
# ).to(device='cuda', non_blocking=True)
# # _ = model.eval()


args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=dev_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


Some weights of the model checkpoint at snunlp/KR-BERT-char16424 were not used when initializing BertForTokenClassification: ['cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at snun

In [19]:
trainer.train()

/home/ktlim/anaconda3/envs/framenet/lib/python3.9/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/ktlim/anaconda3/envs/framenet/lib/python3.9/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Act,Action,Activity,Actor,Addressee,Affected,Agent,Aggregate,Agreement,Artist,Assailant,Assessor,Asset,Attraction,Attribute,Author,Authorities,Authority,Bad Entity,Barrier,Behavior,Benefited Party,Binding Principle,Body Part,Buyer,Captive,Category,Cause,Characteristic,Charges,Child,Chosen,Clothing,Cognizer,Cognizer Agent,Commitment,Communicator,Competition,Competitor,Competitors,Component,Components,Concept 1,Concept 2,Configuration,Connector,Consideration,Container,Containing Object,Content,Contents,Controlling Entity,Convict,Cook,Copy,Cotheme,Count,Court,Created Entity,Creator,Crime,Criteria,Current Instance,Decision,Defender,Deliverer,Dependent Entity,Desirability,Desired State Of Affairs,Destroyer,Difference,Dimension,Direction,Disease,Distance,Distributor,Document,Documents,Donor,Driver,Duration,Duty,Earner,Earnings,Effect,Emission,Employee,Employer,Enemy,Entity,Entity 1,Entity 2,Estimation,Evaluee,Event,Eventuality,Exchanger 1,Exchanger 2,Exchangers,Executed,Expected Event,Experiencer,Exporter,Exporting Area,Fact,Factory,Feature,Field,Figure,Final Category,Final State,Final Value,Financial Instrument,Finding,Firefighter,Fixed Location,Fluid,Focal Occasion,Focal Participant,Form,Fugitive,Goal,Goods,Governed,Grinder,Ground,Group,Harmful Situation,Helper,Hindrance,Hypothetical Event,Idea,Impactee,Impactor,Incident,Individuals,Information,Information Source,Informer,Ingestibles,Ingestor,Initial Category,Inspector,Institution,Instrument,Interlocutor,Interlocutor 1,Interlocutor 2,Interlocutors,Invader,Issue,Item,Items,Land,Leader,Lessee,Lifestyle,Limit1,Limit2,Limits Of Consideration,Location,Location Of Event,Loser,Manner,Mass Theme,Meaning,Means,Medication,Medium,Member,Message,Money,New,New Leader,New Member,New Tool,Noisy Event,Norm,Offense,Offerer,Official,Old,Old Tool,Operator,Opinion,Organization,Origin,Original,Owner,Part,Participant,Participant 1,Participant 2,Participants,Parts,Party 1,Party 2,Path,Patient,Perceiver Agentive,Perpetrator,Phenomenon,Place,Plan,Position,Possession,Possibilities,Potential Hindrance,Potential Recipient,Preventing Cause,Principle,Prison,Prisoner,Problem,Process,Produced Food,Product,Production,Profiled Item,Project,Projectile,Proposal,Proposed Action,Proposition,Protagonist,Protection,Protester,Publisher,Purpose,Question,Range Of Options,Ratifier,Reason,Recipient,Referent,Remainder,Researcher,Resident,Resource,Responding Entity,Response,Responsible Party,Role,Rule,Salient Entity,Scheduled Time,Selector,Self Mover,Seller,Sender,Sentence,Side,Side 1,Side 2,Signatory,Situation,Sought Entity,Sound,Source,Source Emitter,Speaker,Specified Content,Specified Entity,Standard Attribute,Standard Item,State,State Of Affairs,Stimulus,Student,Stuff,Subject,Substance,Supplier,Support,Supported,Supporter,Surrenderer,Survivor,Suspect,System,Teacher,Text,Theme,Theme 1,Theme 2,Themes,Time,Topic,Total,Transport Means,Traveller,Treatment,Trigger,Type,Unconfirmed Content,Undesirable Situation,Unwanted Entity,Useful Location,Value,Value Range,Valued Entity,Variable,Vehicle,Victim,Wearer,Whole,Winner,Work,Wrongdoer,,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,No log,2.738503,"{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 29}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 35}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 166}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 38}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 480}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 17}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 4}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 9}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 24}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5}","{'precis

/home/ktlim/anaconda3/envs/framenet/lib/python3.9/site-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: V seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/home/ktlim/anaconda3/envs/framenet/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/ktlim/anaconda3/envs/framenet/lib/python3.9/site-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=84, training_loss=2.8518019176664806, metrics={'train_runtime': 33.5866, 'train_samples_per_second': 157.741, 'train_steps_per_second': 2.501, 'total_flos': 254249275898016.0, 'train_loss': 2.8518019176664806, 'epoch': 3.0})

In [14]:
trainer.evaluate()

Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 29}" of type <class 'dict'> for key "eval/Act" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 35}" of type <class 'dict'> for key "eval/Action" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 166}" of type <class 'dict'> for key "eval/Activity" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5}" of type <class 'dict'> for key "eval/Actor" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we droppe

{'eval_loss': 2.6181552410125732,
 'eval_Act': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 29},
 'eval_Action': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 35},
 'eval_Activity': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 166},
 'eval_Actor': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 5},
 'eval_Addressee': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 38},
 'eval_Affected': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 16},
 'eval_Agent': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 480},
 'eval_Aggregate': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 7},
 'eval_Agreement': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 17},
 'eval_Artist': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 4},
 'eval_Assailant': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 9},
 'eval_Assessor': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 24},
 'eval_Asset': {'precision': 0.0, 'recall':

In [ ]:
trainer.save_model('snunlp_KR-BERT-char16424.model')

In [ ]:
!tar cvfz snunlp_KR-BERT-char16424_v2.tar.gz snunlp_KR-BERT-char16424.model/

snunlp_KR-BERT-char16424.model/
snunlp_KR-BERT-char16424.model/config.json
snunlp_KR-BERT-char16424.model/training_args.bin
snunlp_KR-BERT-char16424.model/pytorch_model.bin
snunlp_KR-BERT-char16424.model/special_tokens_map.json
snunlp_KR-BERT-char16424.model/tokenizer_config.json
snunlp_KR-BERT-char16424.model/tokenizer.json
snunlp_KR-BERT-char16424.model/vocab.txt


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!mv snunlp_KR-BERT-char16424_v2.tar.gz  /content/drive/MyDrive/

In [ ]:
!rm -rf *.model test-pos *.gz

In [ ]:
### ------- ### ------- ### ------- ### ------- ###
### klue/bert-base
###

In [ ]:
model_checkpoint = "klue/bert-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
dev_tokenized_datasets = dev_dataset.map(tokenize_and_align_labels, batched=True)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=dev_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)
trainer.train()

Map:   0%|          | 0/16228 [00:00<?, ? examples/s]

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the mo

Epoch,Training Loss,Validation Loss,Conj,Dj,Dp,Dv,Erb,Et,Ntj,Onj,Oun,Ropn,Rt,Um,Unct,Ym,,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.100800,0.105164,"{'precision': 0.9375, 'recall': 1.0, 'f1': 0.967741935483871, 'number': 120}","{'precision': 0.9149453219927096, 'recall': 0.93424317617866, 'f1': 0.9244935543278083, 'number': 806}","{'precision': 0.7, 'recall': 0.717948717948718, 'f1': 0.708860759493671, 'number': 39}","{'precision': 0.9605809128630706, 'recall': 0.9706498951781971, 'f1': 0.9655891553701773, 'number': 954}","{'precision': 0.9609275730622617, 'recall': 0.9723561555769848, 'f1': 0.9666080843585237, 'number': 3111}","{'precision': 0.9876543209876543, 'recall': 0.995850622406639, 'f1': 0.9917355371900826, 'number': 482}","{'precision': 0.7849462365591398, 'recall': 0.9240506329113924, 'f1': 0.8488372093023255, 'number': 79}","{'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2}","{'precision': 0.9424360400444939, 'recall': 0.9490338840660879, 'f1': 0.9457234547230361, 'number': 3571}","{'precision': 0.8810572687224669, 'recall': 0.9331259720062208, 'f1': 0.9063444108761329, 'number': 643}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}","{'precision': 0.7586206896551724, 'recall': 0.7457627118644068, 'f1': 0.752136752136752, 'number': 59}","{'precision': 0.25, 'recall': 0.6666666666666666, 'f1': 0.36363636363636365, 'number': 3}","{'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}","{'precision': 0.7941176470588235, 'recall': 0.7714285714285715, 'f1': 0.782608695652174, 'number': 35}",0.941165,0.955890,0.948470,0.971568
2,0.045400,0.090506,"{'precision': 0.9596774193548387, 'recall': 0.9916666666666667, 'f1': 0.9754098360655739, 'number': 120}","{'precision': 0.9507575757575758, 'recall': 0.93424317617866, 'f1': 0.9424280350438049, 'number': 806}","{'precision': 0.6666666666666666, 'recall': 0.717948717948718, 'f1': 0.691358024691358, 'number': 39}","{'precision': 0.9676746611053181, 'recall': 0.9727463312368972, 'f1': 0.9702038682697334, 'number': 954}","{'precision': 0.9744735162731334, 'recall': 0.9816779170684667, 'f1': 0.9780624499599679, 'number': 3111}","{'precision': 0.9876288659793815, 'recall': 0.9937759336099585, 'f1': 0.9906928645294726, 'number': 482}","{'precision': 0.8690476190476191, 'recall': 0.9240506329113924, 'f1': 0.8957055214723927, 'number': 79}","{'precision': 0.5, 'recall': 0.5, 'f1': 0.5, 'number': 2}","{'precision': 0.9585765916041146, 'recall': 0.9655558667040045, 'f1': 0.9620535714285714, 'number': 3571}","{'precision': 0.9433962264150944, 'recall': 0.9331259720062208, 'f1': 0.9382329945269742, 'number': 643}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}","{'precision': 0.9032258064516129, 'recall': 0.9491525423728814, 'f1': 0.9256198347107438, 'number': 59}","{'precision': 0.2857142857142857, 'recall': 0.6666666666666666, 'f1': 0.4, 'number': 3}","{'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1}","{'precision': 0.7272727272727273, 'recall': 0.9142857142857143, 'f1': 0.810126582278481, 'number': 35}",0.960377,0.966387,0.963373,0.980661
3,0.014800,0.096303,"{'precision': 0.9523809523809523, 'recall': 1.0, 'f1': 0.975609756097561, 'number': 120}","{'precision': 0.9473023839397742, 'recall': 0.9367245657568238, 'f1': 0.9419837804117279, 'number': 806}","{'precision': 0.925, 'recall': 0.9487179487179487, 'f1': 0.9367088607594937, 'number': 39}","{'precision': 0.9739311783107404, 'recall': 0.9790356394129979, 'f1': 0.9764767381076842, 'number': 954}","{'precision': 0.97446536865624, 'recall': 0.9813564770170363, 'f1': 0.9778987828315182, 'number': 3111}","{'precision': 0.9876543209876543, 'recall': 0.995850622406639, 'f1': 0.9917355371900826, 'number': 482}","{'precision': 0.8690476190476191, 'recall': 0.9240506329113924, 'f1': 0.8957055214723927, 'number': 79}","{'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2}","{'precision': 0.9596437517394935, 'recall': 0.9655558667040045, 'f1': 0.9625907314349526, 'number': 3571

/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserW

TrainOutput(global_step=3045, training_loss=0.07127448175536784, metrics={'train_runtime': 601.3971, 'train_samples_per_second': 80.952, 'train_steps_per_second': 5.063, 'total_flos': 1377159004655280.0, 'train_loss': 0.07127448175536784, 'epoch': 3.0})

In [ ]:
trainer.evaluate()


Trainer is attempting to log a value of "{'precision': 0.9523809523809523, 'recall': 1.0, 'f1': 0.975609756097561, 'number': 120}" of type <class 'dict'> for key "eval/CONJ" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9473023839397742, 'recall': 0.9367245657568238, 'f1': 0.9419837804117279, 'number': 806}" of type <class 'dict'> for key "eval/DJ" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.925, 'recall': 0.9487179487179487, 'f1': 0.9367088607594937, 'number': 39}" of type <class 'dict'> for key "eval/DP" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9739311783107404, 'recall': 0.9790356394129979, 'f1': 0.9764767381076842, 'numbe

{'eval_loss': 0.09630344808101654,
 'eval_CONJ': {'precision': 0.9523809523809523,
  'recall': 1.0,
  'f1': 0.975609756097561,
  'number': 120},
 'eval_DJ': {'precision': 0.9473023839397742,
  'recall': 0.9367245657568238,
  'f1': 0.9419837804117279,
  'number': 806},
 'eval_DP': {'precision': 0.925,
  'recall': 0.9487179487179487,
  'f1': 0.9367088607594937,
  'number': 39},
 'eval_DV': {'precision': 0.9739311783107404,
  'recall': 0.9790356394129979,
  'f1': 0.9764767381076842,
  'number': 954},
 'eval_ERB': {'precision': 0.97446536865624,
  'recall': 0.9813564770170363,
  'f1': 0.9778987828315182,
  'number': 3111},
 'eval_ET': {'precision': 0.9876543209876543,
  'recall': 0.995850622406639,
  'f1': 0.9917355371900826,
  'number': 482},
 'eval_NTJ': {'precision': 0.8690476190476191,
  'recall': 0.9240506329113924,
  'f1': 0.8957055214723927,
  'number': 79},
 'eval_ONJ': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 2},
 'eval_OUN': {'precision': 0.9596437517394935,
  'reca

In [ ]:
trainer.save_model('klue_bert-base.model')
!tar cvfz klue_bert-base_v2.tar.gz klue_bert-base.model/
!mv klue_bert-base_v2.tar.gz  /content/drive/MyDrive/

Trainer is attempting to log a value of "{'precision': 0.9523809523809523, 'recall': 1.0, 'f1': 0.975609756097561, 'number': 120}" of type <class 'dict'> for key "eval/CONJ" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9473023839397742, 'recall': 0.9367245657568238, 'f1': 0.9419837804117279, 'number': 806}" of type <class 'dict'> for key "eval/DJ" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.925, 'recall': 0.9487179487179487, 'f1': 0.9367088607594937, 'number': 39}" of type <class 'dict'> for key "eval/DP" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9739311783107404, 'recall': 0.9790356394129979, 'f1': 0.9764767381076842, 'numbe

klue_bert-base.model/
klue_bert-base.model/config.json
klue_bert-base.model/training_args.bin
klue_bert-base.model/pytorch_model.bin
klue_bert-base.model/special_tokens_map.json
klue_bert-base.model/tokenizer_config.json
klue_bert-base.model/tokenizer.json
klue_bert-base.model/vocab.txt


In [ ]:
### ------- ### ------- ### ------- ### ------- ###
### koelectra-base-v3
###

In [ ]:
model_checkpoint = "monologg/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
dev_tokenized_datasets = dev_dataset.map(tokenize_and_align_labels, batched=True)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=dev_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()
trainer.evaluate()

Map:   0%|          | 0/16228 [00:00<?, ? examples/s]

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Some weights of the model checkpoint at monologg/koelectra-base-v3-discriminator were not used when initializing ElectraForTokenClassification: ['discriminator_predictions.dense_prediction.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense.bias']
- This IS expected if you are initializing ElectraForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForTokenClassification were not initialized from the model checkpoint at monologg/koelectra-base-v3-discriminator and are newly initialized: ['classifier

Epoch,Training Loss,Validation Loss,Conj,Dj,Dp,Dv,Erb,Et,Ntj,Onj,Oun,Ropn,Rt,Um,Unct,Ym,,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.100100,0.091012,"{'precision': 0.937007874015748, 'recall': 0.9916666666666667, 'f1': 0.9635627530364372, 'number': 120}","{'precision': 0.942643391521197, 'recall': 0.9379652605459057, 'f1': 0.9402985074626866, 'number': 806}","{'precision': 0.574468085106383, 'recall': 0.6923076923076923, 'f1': 0.627906976744186, 'number': 39}","{'precision': 0.9648397104446742, 'recall': 0.9779874213836478, 'f1': 0.9713690786048933, 'number': 954}","{'precision': 0.96844118584635, 'recall': 0.9765348762455802, 'f1': 0.9724711907810498, 'number': 3111}","{'precision': 0.9876796714579056, 'recall': 0.9979253112033195, 'f1': 0.9927760577915377, 'number': 482}","{'precision': 0.8604651162790697, 'recall': 0.9367088607594937, 'f1': 0.8969696969696971, 'number': 79}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}","{'precision': 0.958728388176241, 'recall': 0.9627555306636797, 'f1': 0.9607377392762331, 'number': 3571}","{'precision': 0.9430379746835443, 'recall': 0.926905132192846, 'f1': 0.9349019607843138, 'number': 643}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}","{'precision': 0.8382352941176471, 'recall': 0.9661016949152542, 'f1': 0.8976377952755905, 'number': 59}","{'precision': 0.14285714285714285, 'recall': 0.3333333333333333, 'f1': 0.2, 'number': 3}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}","{'precision': 0.55, 'recall': 0.6285714285714286, 'f1': 0.5866666666666667, 'number': 35}",0.955346,0.963157,0.959236,0.978391
2,0.047500,0.086577,"{'precision': 0.975609756097561, 'recall': 1.0, 'f1': 0.9876543209876543, 'number': 120}","{'precision': 0.9519595448798989, 'recall': 0.93424317617866, 'f1': 0.9430181590482153, 'number': 806}","{'precision': 0.8780487804878049, 'recall': 0.9230769230769231, 'f1': 0.9, 'number': 39}","{'precision': 0.9700413223140496, 'recall': 0.9842767295597484, 'f1': 0.9771071800208118, 'number': 954}","{'precision': 0.9744408945686901, 'recall': 0.9803921568627451, 'f1': 0.977407466752123, 'number': 3111}","{'precision': 0.985655737704918, 'recall': 0.9979253112033195, 'f1': 0.9917525773195875, 'number': 482}","{'precision': 0.9024390243902439, 'recall': 0.9367088607594937, 'f1': 0.9192546583850932, 'number': 79}","{'precision': 0.6666666666666666, 'recall': 1.0, 'f1': 0.8, 'number': 2}","{'precision': 0.967606813739179, 'recall': 0.9703164379725567, 'f1': 0.9689597315436241, 'number': 3571}","{'precision': 0.9533762057877814, 'recall': 0.9222395023328149, 'f1': 0.9375494071146245, 'number': 643}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}","{'precision': 0.9180327868852459, 'recall': 0.9491525423728814, 'f1': 0.9333333333333333, 'number': 59}","{'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1': 0.3333333333333333, 'number': 3}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}","{'precision': 0.8333333333333334, 'recall': 0.8571428571428571, 'f1': 0.8450704225352113, 'number': 35}",0.966865,0.969012,0.967937,0.981511
3,0.018300,0.078612,"{'precision': 0.975609756097561, 'recall': 1.0, 'f1': 0.9876543209876543, 'number': 120}","{'precision': 0.9584382871536524, 'recall': 0.9441687344913151, 'f1': 0.95125, 'number': 806}","{'precision': 0.8372093023255814, 'recall': 0.9230769230769231, 'f1': 0.878048780487805, 'number': 39}","{'precision': 0.9780564263322884, 'recall': 0.9811320754716981, 'f1': 0.9795918367346939, 'number': 954}","{'precision': 0.9772872680742163, 'recall': 0.9819993571198972, 'f1': 0.9796376463043129, 'number': 3111}","{'precision': 0.985655737704918, 'recall': 0.9979253112033195, 'f1': 0.9917525773195875, 'number': 482}","{'precision': 0.8020833333333334, 'recall': 0.9746835443037974, 'f1': 0.88, 'number': 79}","{'precision': 0.6666666666666666, 'recall': 1.0, 'f1': 0.8, 'number': 2}","{'precision': 0.9693080357142857, 'recall': 0.972836740408849, 'f1': 0.971

/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserW

Trainer is attempting to log a value of "{'precision': 0.975609756097561, 'recall': 1.0, 'f1': 0.9876543209876543, 'number': 120}" of type <class 'dict'> for key "eval/CONJ" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9584382871536524, 'recall': 0.9441687344913151, 'f1': 0.95125, 'number': 806}" of type <class 'dict'> for key "eval/DJ" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8372093023255814, 'recall': 0.9230769230769231, 'f1': 0.878048780487805, 'number': 39}" of type <class 'dict'> for key "eval/DP" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9780564263322884, 'recall': 0.9811320754716981, 'f1': 0.9795918367346939, 'numb

{'eval_loss': 0.07861168682575226,
 'eval_CONJ': {'precision': 0.975609756097561,
  'recall': 1.0,
  'f1': 0.9876543209876543,
  'number': 120},
 'eval_DJ': {'precision': 0.9584382871536524,
  'recall': 0.9441687344913151,
  'f1': 0.95125,
  'number': 806},
 'eval_DP': {'precision': 0.8372093023255814,
  'recall': 0.9230769230769231,
  'f1': 0.878048780487805,
  'number': 39},
 'eval_DV': {'precision': 0.9780564263322884,
  'recall': 0.9811320754716981,
  'f1': 0.9795918367346939,
  'number': 954},
 'eval_ERB': {'precision': 0.9772872680742163,
  'recall': 0.9819993571198972,
  'f1': 0.9796376463043129,
  'number': 3111},
 'eval_ET': {'precision': 0.985655737704918,
  'recall': 0.9979253112033195,
  'f1': 0.9917525773195875,
  'number': 482},
 'eval_NTJ': {'precision': 0.8020833333333334,
  'recall': 0.9746835443037974,
  'f1': 0.88,
  'number': 79},
 'eval_ONJ': {'precision': 0.6666666666666666,
  'recall': 1.0,
  'f1': 0.8,
  'number': 2},
 'eval_OUN': {'precision': 0.969308035714285

In [ ]:
trainer.save_model('koelectra-base.model')
!tar cvfz koelectra-base_v2.tar.gz koelectra-base.model/
!mv koelectra-base_v2.tar.gz  /content/drive/MyDrive/

koelectra-base.model/
koelectra-base.model/config.json
koelectra-base.model/training_args.bin
koelectra-base.model/pytorch_model.bin
koelectra-base.model/special_tokens_map.json
koelectra-base.model/tokenizer_config.json
koelectra-base.model/tokenizer.json
koelectra-base.model/vocab.txt


In [ ]:
!rm -rf *.model test-pos *.gz

In [ ]:
# bert-base-multilingual-cased

model_checkpoint = "bert-base-multilingual-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

train_tokenized_datasets = train_dataset.map(tokenize_and_align_labels, batched=True)
dev_tokenized_datasets = dev_dataset.map(tokenize_and_align_labels, batched=True)

model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

args = TrainingArguments(
    f"test-{task}",
    evaluation_strategy = "epoch",
    learning_rate=1e-4,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=1e-5,
)

data_collator = DataCollatorForTokenClassification(tokenizer)
metric = load_metric("seqeval")


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [[label_list[p] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]
    true_labels = [[label_list[l] for (p, l) in zip(prediction, label) if l != -100] for prediction, label in zip(predictions, labels)]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return results
    
trainer = Trainer(
    model,
    args,
    train_dataset=train_tokenized_datasets,
    eval_dataset=dev_tokenized_datasets,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)


trainer.train()
trainer.evaluate()


Map:   0%|          | 0/16228 [00:00<?, ? examples/s]

Map:   0%|          | 0/1418 [00:00<?, ? examples/s]

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForTokenClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at 

Epoch,Training Loss,Validation Loss,Conj,Dj,Dp,Dv,Erb,Et,Ntj,Onj,Oun,Ropn,Rt,Um,Unct,Ym,,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.191400,0.155804,"{'precision': 0.9022556390977443, 'recall': 1.0, 'f1': 0.9486166007905138, 'number': 120}","{'precision': 0.9217506631299734, 'recall': 0.8622828784119106, 'f1': 0.891025641025641, 'number': 806}","{'precision': 0.575, 'recall': 0.5897435897435898, 'f1': 0.5822784810126582, 'number': 39}","{'precision': 0.894, 'recall': 0.9371069182389937, 'f1': 0.9150460593654043, 'number': 954}","{'precision': 0.9427100096556164, 'recall': 0.9414979106396657, 'f1': 0.9421035702798327, 'number': 3111}","{'precision': 0.967479674796748, 'recall': 0.9875518672199171, 'f1': 0.9774127310061602, 'number': 482}","{'precision': 0.8148148148148148, 'recall': 0.8354430379746836, 'f1': 0.825, 'number': 79}","{'precision': 1.0, 'recall': 0.5, 'f1': 0.6666666666666666, 'number': 2}","{'precision': 0.903486750348675, 'recall': 0.9070288434612154, 'f1': 0.9052543320290666, 'number': 3571}","{'precision': 0.810930576070901, 'recall': 0.8538102643856921, 'f1': 0.8318181818181819, 'number': 643}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}","{'precision': 0.7846153846153846, 'recall': 0.864406779661017, 'f1': 0.8225806451612904, 'number': 59}","{'precision': 0.2857142857142857, 'recall': 0.6666666666666666, 'f1': 0.4, 'number': 3}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}","{'precision': 0.5142857142857142, 'recall': 0.5142857142857142, 'f1': 0.5142857142857142, 'number': 35}",0.908389,0.914808,0.911587,0.955474
2,0.100700,0.138596,"{'precision': 0.9230769230769231, 'recall': 1.0, 'f1': 0.9600000000000001, 'number': 120}","{'precision': 0.8798017348203222, 'recall': 0.8808933002481389, 'f1': 0.8803471791692499, 'number': 806}","{'precision': 0.6744186046511628, 'recall': 0.7435897435897436, 'f1': 0.7073170731707318, 'number': 39}","{'precision': 0.9284232365145229, 'recall': 0.9381551362683438, 'f1': 0.9332638164754953, 'number': 954}","{'precision': 0.9460408684546615, 'recall': 0.9524268723882996, 'f1': 0.9492231299054942, 'number': 3111}","{'precision': 0.9754601226993865, 'recall': 0.9896265560165975, 'f1': 0.9824922760041194, 'number': 482}","{'precision': 0.8461538461538461, 'recall': 0.8354430379746836, 'f1': 0.840764331210191, 'number': 79}","{'precision': 1.0, 'recall': 0.5, 'f1': 0.6666666666666666, 'number': 2}","{'precision': 0.9219052893935198, 'recall': 0.9322318678241389, 'f1': 0.927039821776664, 'number': 3571}","{'precision': 0.8742138364779874, 'recall': 0.864696734059098, 'f1': 0.8694292415949961, 'number': 643}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 2}","{'precision': 0.859375, 'recall': 0.9322033898305084, 'f1': 0.8943089430894309, 'number': 59}","{'precision': 0.125, 'recall': 0.6666666666666666, 'f1': 0.21052631578947367, 'number': 3}","{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'number': 1}","{'precision': 0.6122448979591837, 'recall': 0.8571428571428571, 'f1': 0.7142857142857143, 'number': 35}",0.921457,0.931967,0.926682,0.963938
3,0.041700,0.125147,"{'precision': 0.9448818897637795, 'recall': 1.0, 'f1': 0.97165991902834, 'number': 120}","{'precision': 0.9217948717948717, 'recall': 0.892059553349876, 'f1': 0.9066834804539723, 'number': 806}","{'precision': 0.9230769230769231, 'recall': 0.9230769230769231, 'f1': 0.9230769230769231, 'number': 39}","{'precision': 0.9460020768431984, 'recall': 0.9549266247379455, 'f1': 0.9504434011476266, 'number': 954}","{'precision': 0.9560333761232349, 'recall': 0.9575699132111861, 'f1': 0.9568010277822386, 'number': 3111}","{'precision': 0.9696356275303644, 'recall': 0.9937759336099585, 'f1': 0.9815573770491803, 'number': 482}","{'precision': 0.8536585365853658, 'recall': 0.8860759493670886, 'f1': 0.8695652173913043, 'number': 79}","{'precision': 1.0, 'recall': 0.5, 'f1': 0.6666666666666666, 'number': 2}","{'precision': 0.9365962180200222, 'recall': 0.9431531783814058, 'f1': 0.9398

/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NOUN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: NUM seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: ADJ seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: VERB seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: PROPN seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.9/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserW

Trainer is attempting to log a value of "{'precision': 0.9448818897637795, 'recall': 1.0, 'f1': 0.97165991902834, 'number': 120}" of type <class 'dict'> for key "eval/CONJ" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9217948717948717, 'recall': 0.892059553349876, 'f1': 0.9066834804539723, 'number': 806}" of type <class 'dict'> for key "eval/DJ" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9230769230769231, 'recall': 0.9230769230769231, 'f1': 0.9230769230769231, 'number': 39}" of type <class 'dict'> for key "eval/DP" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9460020768431984, 'recall': 0.9549266247379455, 'f1': 0.9504434011476

{'eval_loss': 0.12514713406562805,
 'eval_CONJ': {'precision': 0.9448818897637795,
  'recall': 1.0,
  'f1': 0.97165991902834,
  'number': 120},
 'eval_DJ': {'precision': 0.9217948717948717,
  'recall': 0.892059553349876,
  'f1': 0.9066834804539723,
  'number': 806},
 'eval_DP': {'precision': 0.9230769230769231,
  'recall': 0.9230769230769231,
  'f1': 0.9230769230769231,
  'number': 39},
 'eval_DV': {'precision': 0.9460020768431984,
  'recall': 0.9549266247379455,
  'f1': 0.9504434011476266,
  'number': 954},
 'eval_ERB': {'precision': 0.9560333761232349,
  'recall': 0.9575699132111861,
  'f1': 0.9568010277822386,
  'number': 3111},
 'eval_ET': {'precision': 0.9696356275303644,
  'recall': 0.9937759336099585,
  'f1': 0.9815573770491803,
  'number': 482},
 'eval_NTJ': {'precision': 0.8536585365853658,
  'recall': 0.8860759493670886,
  'f1': 0.8695652173913043,
  'number': 79},
 'eval_ONJ': {'precision': 1.0,
  'recall': 0.5,
  'f1': 0.6666666666666666,
  'number': 2},
 'eval_OUN': {'prec

In [ ]:
trainer.save_model('bert-base-multilingual-cased.model')
!tar cvfz bert-base-multilingual-cased.tar.gz bert-base-multilingual-cased.model/
!mv bert-base-multilingual-cased.tar.gz  /content/drive/MyDrive/


bert-base-multilingual-cased.model/
bert-base-multilingual-cased.model/config.json
bert-base-multilingual-cased.model/training_args.bin
bert-base-multilingual-cased.model/pytorch_model.bin
bert-base-multilingual-cased.model/special_tokens_map.json
bert-base-multilingual-cased.model/tokenizer_config.json
bert-base-multilingual-cased.model/tokenizer.json
bert-base-multilingual-cased.model/vocab.txt
